In [1]:
%load_ext autoreload
%autoreload 2

import os 
os.chdir('../generative')

In [2]:
from collections import namedtuple

Args = namedtuple('Args', [
    'train_dataset',
    'valid_dataset',
    'batch_size',
    'learning_rate',
    'max_epoch',
    'model'])

args = Args('../data/train', '../data/val', 1, 1e-4, 70, 'VAE')

In [5]:
import argparse
import time
import models
import training_loss

import torch
from dataset_manager import GenerativeAIDataset
from torch.utils.data import DataLoader

from transformation.data_preprocessing import get_data_composing
from training_loss import get_loss_train


In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 1 # args.batch_size
model_name = args.model



data_processing = get_data_composing(model_name)

train_dataset = GenerativeAIDataset(args.train_dataset,
                            data_processing,
                            class_c='BOAFAB')

In [7]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, drop_last=True)

In [8]:
sample = next(iter(train_dataloader))

In [10]:
sample['samples'].shape

torch.Size([1, 1, 16000])

In [11]:
model = models.create_model(model_name=model_name).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [8]:
sample = next(iter(train_dataloader))

In [12]:
loss_function = training_loss.get_loss_train(model_name)

In [13]:
def train(num_epochs):
    for epoch in range(num_epochs):
        for step, batch in enumerate(train_dataloader):
            batch = batch['samples'].to(device)
            optimizer.zero_grad()
            recon_batch, mu, logvar = model(batch)
            loss = loss_function(model, batch)
            loss.backward()
            optimizer.step()
            if epoch % 5 == 0 and step == 0:
                print(f"Epoch {epoch} | step {step:03d} Loss: {loss.item()} ")

In [14]:
train(1)

KeyboardInterrupt: 

In [16]:
def sample(num_samples):
    with torch.no_grad():
        z = torch.randn(num_samples, model.latent_dim).to(device)
        samples = model.decode(z)

    return samples

In [21]:
samples

tensor([[0.5244, 0.7477, 0.7399,  ..., 0.7539, 0.7512, 0.5170],
        [0.5195, 0.7468, 0.7591,  ..., 0.7457, 0.7621, 0.5200],
        [0.5245, 0.7487, 0.7436,  ..., 0.7477, 0.7538, 0.5203],
        ...,
        [0.5203, 0.7520, 0.7419,  ..., 0.7500, 0.7468, 0.5210],
        [0.5257, 0.7465, 0.7532,  ..., 0.7497, 0.7499, 0.5192],
        [0.5206, 0.7509, 0.7453,  ..., 0.7502, 0.7452, 0.5199]])

In [22]:
import soundfile as sf

samples = sample(10)
sr = BITRATE
for n, s in enumerate(samples):
    to_save_audio = sf.write(f'../data/generated/BOAFAB/gen_{n}.wav', s, sr)

In [37]:
h = sample(1).cpu().numpy()[0]
plt.figure().set_figwidth(12)
librosa.display.waveshow(h, sr=BITRATE)
# Perform a Short-Time Fourier Transform (STFT)
D = librosa.stft(h)
# Convert the complex-valued result to magnitude
S_db = librosa.amplitude_to_db(abs(D), ref=np.max)
# Display the spectrogram
plt.figure(figsize=(14, 5))
librosa.display.specshow(S_db, sr=BITRATE, x_axis='time', y_axis='log')
# plt.colorbar(format='%+2.0f dB')
plt.title('Espectrograma de Frecuencia Real')
plt.show()
Audio(data=h , rate=BITRATE)

NameError: name 'sample' is not defined